In [16]:
from utils.df_handle import *

In [17]:
URL_admin ='https://docs.google.com/spreadsheets/d/e/2PACX-1vSYWdHL8hh3RoQTe4dAaidvJ83sbrKiZm9gseWgNru2uWRtfpKzYY87ix9rjl5AK5F7Fa3VGpGA4QgQ/pub?gid=206442339&single=true&output=csv'

In [18]:
df = pd.read_csv(URL_admin,header = 0)
df.columns = cleancols(df)
df.columns = lower_col(df)

In [19]:
# Dùng để mapping tuần sang ngày
data_convert = pd.read_csv("convert_date.csv", dayfirst=True, parse_dates=['ngay'])
reference = pd.DataFrame(data_convert)

In [20]:
df_unpivoted = df.melt(id_vars=['makhcu','manv','tenkhachhang','tennhanvien','giaiphap'], 
                       var_name='tuan', value_name='thongtinthanhtoan')
df3 = pd.DataFrame(df_unpivoted)

In [21]:
df4=df3.merge(reference, on='tuan', how='left')
df5=pd.DataFrame(df4)

In [93]:
df5.rename({'ngay':'thoigiangoi'}
          ,axis='columns',inplace =True)
df5['updated_at'] = datetime.now()

In [94]:
df5['thoigiangoi'] = pd.to_datetime(df5['thoigiangoi'], format='%Y%m%d %H%M%S')


In [95]:
df6=df5[['manv','thongtinthanhtoan','giaiphap','thoigiangoi','makhcu','updated_at']]

In [96]:
df7=df6.dropna(subset=['manv','thongtinthanhtoan'])

In [99]:
def upsert_table():
	df = df7
	pk = ['manv','makhcu','thongtinthanhtoan']
	execute_values_upsert(df, "d_tttt_ins",pk)

In [100]:
def main():  
    upsert_table()

In [101]:
if __name__ == "__main__":
    main()

INSERT INTO d_tttt_ins(manv,thongtinthanhtoan,giaiphap,thoigiangoi,inserted_at,makhcu) VALUES %s ON CONFLICT (manv, makhcu, thoigiangoi) DO UPDATE SET thongtinthanhtoan = excluded.thongtinthanhtoan, giaiphap = excluded.giaiphap, inserted_at = excluded.inserted_at
Data inserted using execute_values() successfully..
